# Linear Regression with Spark

In [20]:
# Import modules and start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()
# Read in the data
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
# Inspect the data
# This is the format spark needs for machine learning
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

## Instantiate the model and fit the model to the data

In [21]:
# Create the model
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
# Fit the model to the data
lrModel = lr.fit(training)

## Evaluate the model and grab statistics

In [22]:
lrModel.coefficients
lrModel.intercept

training_summary = lrModel.summary
training_summary.r2

0.027839179518600154

## Train-Test-Split

In [25]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')
# Creates a list of two dataframes
# split_object = all_data.randomSplit([0.7,0.3])
# Using tuple unpacking
train_data,test_data = all_data.randomSplit([0.7,0.3])
# 349 observations in train
train_data.describe().show()
# 152 observations in test
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                350|
|   mean| 0.6186245063611274|
| stddev| 10.437661082455055|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                350|
|   mean| 0.6186245063611274|
| stddev| 10.437661082455055|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [27]:
# Create the model
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
# fit the model to the data
lrmodel = lr.fit(train_data)
# evaluate the model results to the test_data
test_results = lrmodel.evaluate(test_data)

In [30]:
# Evaluate the model
test_results.residuals.show()
test_results.rootMeanSquaredError

+-------------------+
|          residuals|
+-------------------+
|-27.520961802997125|
| -27.47714650853331|
| -18.36765118576825|
|-18.550635917844605|
|-21.556404770509236|
|-16.437325709003623|
| -17.23108258374071|
| -15.24463415172983|
|-17.764381022776096|
|-13.463643406795878|
|-15.611090149989883|
|-16.243951524871555|
|-12.371580392745074|
|-14.834480548873126|
|-17.759677070389863|
| -9.853037675729993|
| -14.20612711233753|
|-16.199643047823542|
| -14.89377053758816|
|-12.783246846810355|
+-------------------+
only showing top 20 rows



10.14744906376347

## Deploy model

In [31]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lrmodel.transform(unlabeled_data)
predictions.show()